In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm # import statsmodels 

/usr/local/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read in the data
df = pd.read_csv("./../data/changes.csv")

In [3]:
df.set_index('date', inplace=True)
print(df.shape)
df.head()

(7115050, 38)


,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
# try removing the averages so as to not include endogenous variables on the right hand side
df.drop(['eurusd', 'vehicles'], axis=1, inplace=True)
print(df.shape)
df.head()

(7115050, 36)


,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
# replace the oil prices for the last 30 days with the predictions
oil = pd.read_csv('./../data/linear_oil_yhat.csv')

last_30 = ['2015-11-12', '2015-11-13', '2015-11-14', '2015-11-15', '2015-11-16', '2015-11-17', '2015-11-18',
          '2015-11-19', '2015-11-20', '2015-11-21', '2015-11-22', '2015-11-23', '2015-11-24', '2015-11-25',
          '2015-11-26', '2015-11-27', '2015-11-28', '2015-11-29', '2015-11-30', '2015-12-01', '2015-12-02',
          '2015-12-03', '2015-12-04', '2015-12-05', '2015-12-06', '2015-12-07', '2015-12-08', '2015-12-09',
          '2015-12-10', '2015-12-10']

for index, date in enumerate(last_30):
    df.loc[date, 'rotterdam'] = oil['rot_yhat'][index]
    df.loc[date, 'brent'] = oil['brent_yhat'][index]
    df.loc[date, 'wti'] = oil['wti_yhat'][index]

In [6]:
# now order needs to be by day rather than by station -- reorder by num_days
df.sort_values(by=['num_days', 'station'], inplace=True)
df.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-16,1.537824,53.519798,10.0004,538.870667,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-16,1.588412,48.894600,10.0005,12108.902344,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-16,1.607947,49.911800,10.0018,270.597382,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-16,1.545250,49.793301,10.0023,3399.334473,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# split into train and test sets
day_30 = df['num_days'].unique()[-30]
train = df.loc[df['num_days'] < day_30]
test = df.loc[df['num_days'] >= day_30]

In [8]:
train.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-16,1.537824,53.519798,10.0004,538.870667,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-16,1.588412,48.894600,10.0005,12108.902344,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-16,1.607947,49.911800,10.0018,270.597382,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-16,1.545250,49.793301,10.0023,3399.334473,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# split into input and outputs
train_X, train_y = train[train.columns.difference(['e5gas', 'station'])], train['e5gas']
test_X, test_y = test[test.columns.difference(['e5gas', 'station'])], test['e5gas']
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(6743830, 34) (6743830,) (371220, 34) (371220,)


In [10]:
# fit model
X = sm.add_constant(train_X) ## let's add an intercept (beta_0) to our model

# Note the difference in argument order
model = sm.OLS(train_y, X).fit() ## sm.OLS(output, input)
# predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  e5gas   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                 1.912e+06
Date:                Sun, 29 Apr 2018   Prob (F-statistic):               0.00
Time:                        19:39:54   Log-Likelihood:             1.3745e+07
No. Observations:             6743830   AIC:                        -2.749e+07
Df Residuals:                 6743795   BIC:                        -2.749e+07
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8937      0.001    766.120      0.000       0.891       0.896
aral           0.0382   3.41e-05   1121.815      0.000       0.038       0.038
autobahn       0.0517   7.05e-05    733.447      0.000       0.052       0.052
brent         -0.0001   3.91e-06    -27.334      0.000      -0.000   -9.92e-05
dautobahn  -2.313e-08   1.63e-09    -14.191      0.000   -2.63e-08   -1.99e-08
esso           0.0127   4.74e-05    267.255      0.000       0.013       0.013
jet           -0.0093   6.09e-05   -152.031      0.000      -0.009      -0.009
latitude       0.0002   2.35e-05      8.270      0.000       0.000       0.000
longitude     -0.0010   1.64e-05    -58.277      0.000      -0.001      -0.001
num_days       0.0001   1.88e-07    654.510      0.000       0.000       0.000
rotterdam      0.8815      0.000   3029.568      0.000       0.881       0.882
shell          0.0360   3.77e-05    954.339      0.000       0.036       0.036
state_1        0.0023      6e-05     37.525      0.000       0.002       0.002
state_10      -0.0112   7.21e-05   -155.220      0.000      -0.011      -0.011
state_11       0.0010      0.000      7.929      0.000       0.001       0.001
state_12       0.0048      0.000     42.280      0.000       0.005       0.005
state_13      -0.0006      0.000     -4.724      0.000      -0.001      -0.000
state_14      -0.0130      0.000    -89.484      0.000      -0.013      -0.013
state_15       0.0036      0.000     34.971      0.000       0.003       0.004
state_2       -0.0114      0.000    -76.428      0.000      -0.012      -0.011
state_3        0.0058      0.000     41.016      0.000       0.006       0.006
state_4       -0.0260      0.000   -118.377      0.000      -0.026      -0.026
state_5       -0.0203      0.000   -127.979      0.000      -0.021      -0.020
state_6       -0.0037   6.85e-05    -54.242      0.000      -0.004      -0.004
state_7       -0.0068      0.000    -64.238      0.000      -0.007      -0.007
state_8       -0.0037      0.000    -23.279      0.000      -0.004      -0.003
state_9       -0.0104   8.15e-05   -127.139      0.000      -0.011      -0.010
total          0.0275   7.95e-05    345.899      0.000       0.027       0.028
weekday_1  -7.669e-05   4.54e-05     -1.690      0.091      -0.000    1.22e-05
weekday_2      0.0006   4.54e-05     13.983      0.000       0.001       0.001
weekday_3     -0.0018   4.56e-05    -40.227      0.000      -0.002      -0.002
weekday_4     -0.0024   4.55e-05    -52.714      0.000      -0.002      -0.002
weekday_5     -0.0025   4.54e-05    -55.383      0.000      -0.003      -0.002
weekday_6     -0.0012   4.54e-05    -27.133      0.000      -0.001      -0.001
wti            0.0013    3.8e-06    353.447      0.000       0.001       0.001
==============================================================================
Om

In [11]:
# make a prediction
test_X_beta = sm.add_constant(test_X)
yhat = model.predict(test_X_beta)

In [12]:
# calculate RMSE
rmse = sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.035


In [17]:
test_X['y'] = test_y

In [18]:
test_X.head()

,aral,autobahn,brent,dautobahn,esso,jet,latitude,longitude,num_days,rotterdam,...,total,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,wti,yhat,y
date,,,,,,,,,,,,,,,,,,,,,
2015-11-12,0,0,48.112483,14850.392578,0,0,51.157501,10.0002,545,0.410123,...,0,0,0,0,1,0,0,44.014304,1.370096,1.324000
2015-11-12,0,0,48.112483,538.870667,0,1,53.519798,10.0004,545,0.410123,...,0,0,0,0,1,0,0,44.014304,1.345055,1.308167
2015-11-12,0,0,48.112483,12108.902344,0,0,48.894600,10.0005,545,0.410123,...,0,0,0,0,1,0,0,44.014304,1.373437,1.363167
2015-11-12,0,0,48.112483,270.597382,0,0,49.911800,10.0018,545,0.410123,...,0,0,0,0,1,0,0,44.014304,1.412178,1.412750
2015-11-12,0,0,48.112483,3399.334473,0,0,49.793301,10.0023,545,0.410123,...,0,0,0,0,1,0,0,44.014304,1.376062,1.392750


In [19]:
test_X.to_csv("linear_predictions.csv")

In [21]:
type(model.summary())

method

In [64]:
# each day to make a graph of the error
ob = pd.DataFrame(inv_y)
pred = pd.DataFrame(inv_yhat)
pred[15] = round(pred[15])
pred[16] = round(pred[16])

In [45]:
rmse_days = []
days = []
for i in range(545,575):
    c_ob = ob[ob[16] == i][0]
    c_pred = pred[pred[16] == i][0]
    rmse_days.append(sqrt(mean_squared_error(c_ob, c_pred)))
    days.append(i)
    

In [50]:
rmse_days
days_df = pd.DataFrame()
days_df['rmse'] = rmse_days
days_df['date'] = last_30
days_df['date'] = pd.to_datetime(days_df['date'])

In [52]:
from bokeh.plotting import figure, output_file, show

output_file("line.html")

p = figure(plot_width=400, plot_height=400, x_axis_type='datetime')
p.yaxis.axis_label = 'Root Mean Squared Error'

# add a line renderer
p.line(days_df['date'], days_df['rmse'], line_width=2)

show(p)

In [62]:
print(ob[ob[16] == 546][0].mean())
print(pred[pred[16] == 546][0].mean())

1.3557571172714233
1.3972167226620094


In [68]:
gas_201 = pred[pred[15] == 201][0]

In [70]:
output_file("stuff.html")

p = figure(plot_width=400, plot_height=400, x_axis_type='datetime')
p.yaxis.axis_label = 'Gasoline Price'

# add a line renderer
p.line(days_df['date'], gas_201, line_width=2, color='green')

show(p)